# Project 5: Finite Volume Method

<h1 style="position: absolute; display: flex; flex-grow: 0; flex-shrink: 0; flex-direction: row-reverse; top: 90px;right: 30px; margin: 0; border: 0">
    <style>
        .markdown {width:100%; position: relative}
        article { position: relative }
    </style>
    <img src="https://gitlab.tudelft.nl/mude/public/-/raw/main/tu-logo/TU_P1_full-color.png" style="width:100px" />
    <img src="https://gitlab.tudelft.nl/mude/public/-/raw/main/mude-logo/MUDE_Logo-small.png" style="width:100px" />
</h1>
<h2 style="height: 25px">
</h2>

*[CEGM1000 MUDE](http://mude.citg.tudelft.nl/): Week 2.1, Friday, November 17, 2023.*

## Overview:

From the [conservation laws covered in the textbook](https://mude.citg.tudelft.nl/book/fvm/getting_started.html#conservation-laws), recall that diffusion can be described for a scalar quantity $\phi(\mathbf{x},t)$ as follows:

$$
\frac{\partial \phi(\mathbf{x},t)}{\partial t} = D\nabla^2 \phi(\mathbf{x},t)
$$

where, $\nabla^2$ is the Laplacian operator. The 1D equation for diffusion of a scalar quantity $\phi(x,t)$ is thus:

$$\frac{\partial \phi(x,t)}{\partial t}= D \frac{\partial^2 \phi (x,t)}{\partial x^2}$$

with $D$ as the diffusivity. In this project we will discretise and solve the diffusion equation in 1D and 2D using FVM, as well as evaluate stability as we did with advection. In addition, Dirichlet boundary conditions will need to be applied; a Dirichlet condition is one where the boundary is specified the quantity of interest $\phi$ (as apposed to $\nabla \phi$, which is a Neumann condition). This can be easily applied by specifying the value of the East, West, North or South face of the boundary volume, as needed. Note that the "width" over which the gradient term will be half of the volume width in this case. In the code provided here boundary conditions are computed at the appropriate face for each time increment, then the remaining (interior) volumes are computed using for loops.

A von Neumann analysis of the diffusion equation suggests that stability occurs when the Courant number is as follows:

$$\frac{D\Delta t}{(\Delta x)^2} \leq \frac{1}{2}$$

Which can also be applied for the $y$-direction. Recall that diffusivity has units of L$^2$/T.

<div style="background-color:#AABAB2; color: black; vertical-align: middle; width:95%; padding:15px; margin: 10px; border-radius: 10px">
<p>
<b>Task 0:</b> 
    
Tasks 1 and 2 below focus on deriving and implementing the 1D and 2D solution for the diffusion equation. The boundary conditions and initial conditions are purposefully unspecified in the text to force you to identify them from the code (and think of the application yourself for one of the answers in the report). After you have completed these tasks don't forget to explore the stability of the numerical solution and fill in the answers to this project as requested in <code>Report.md</code>. 

</p>
</div>


In [1]:
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
from ipywidgets import interact, fixed, widgets
import matplotlib.pyplot as plt

## Task 1: 1D Implementation

The code below is set up to solve the 1D diffusion equation. It works more or less in a similar way to the Wednesday assignment, although one difference is the boundary conditions and initial conditions. The problem is set up such that the initial condition is zero inside between the left and right boundary conditions, which are specified via the list `pBC`as the left and right boundary, respectively.

In the code below you will have to implement the boundary conditions that are solved for in each time step. You can do this by using the left and right boundary for the left and right face of the left- and right-most finite volume in the system in place of the averaged West and East faces.

<div style="background-color:#AABAB2; color: black; vertical-align: middle; width:95%; padding:15px; margin: 10px; border-radius: 10px">
<p>
<b>Task 1:</b> 
    
The code below is set up to solve the 1D diffusion equation. Read through the code to identify the boundary conditions and initial conditions. Then derive the solution for the quantity of interest $\phi_i^{n+1}$ and implement it in the code. Note: you will need to apply the boundary condition <code>pBC</code>! This requires defining the left and right side of the volume outside of the spatial loop.

</p>
</div>


In [2]:
def diffusion_1D(p, dx, Nx, D, dt, pBC):
    """FVM diffusion, 1D.
    
    F_? refers to phi on the face of a volume.
    
    Other variable names should be obvious, based on previous assignment.
    """ 

    F_e = np.zeros(Nx)
    F_w = np.zeros(Nx)
    
    # Face values for boundary volumes
    F_w[0] = (pBC[0]-p[0])/(0.5*dx)
    F_e[-1] = (pBC[1]-p[-1])/(0.5*dx)
    
    #Face values for interior volumes
    for i in range(1, Nx - 1): 
        F_w[i] = (p[i] - p[i-1]) / (dx)
        F_e[i] = (p[i+1] - p[i]) / (dx)
        
    # compute phi for all volumes
    p_new = p + ((D * dt) / dx) * (F_e - F_w)
    return p_new

def FVM_plot(x, u, step):
    fig = plt.figure()
    ax = plt.axes(xlim=(0, 300), ylim=(0, 100)) 
    ax.plot(x, u[step], marker='.')
    plt.xlabel('Location')
    plt.ylabel('Temperature')
    plt.show()
    
def check_variables_1D():
    print('Current variables values:')
    print(f'  D  [m^2/s]: {D:0.2f}')
    print(f'  L   [ m ]: {L:0.1f}')
    print(f'  Nx  [---]: {Nx:0.1f}')
    print(f'  T   [ s ]: {T:0.1f}')
    print(f'  Nt  [---]: {Nt:0.1f}')
    print(f'  dx  [ m ]: {dx:0.2e}')
    print(f'  dt  [ s ]: {dt:0.2e}')
    print(f'Courant, C_N, direction x: {D*dt/dx**2:.2e}')

In [3]:
L = 300
Nx = 10
D = 1
dx = L/Nx

pBC = [50, 100]

x = np.linspace(dx/2, L-dx/2, Nx)

T = 40000
Nt = 200
dt = T/Nt

p_all = np.zeros((Nt+1, Nx))
p_all[0, 0] = pBC[0]
p_all[0, -1] = pBC[1]

check_variables_1D()

for t in range(Nt):  
    p = diffusion_1D(p_all[t], dx, Nx, D, dt, pBC)
    p_all[t+1] = p

play = widgets.Play(min=0, max= Nt -1, step=1, value=0, interval=100, disabled=False)
slider = widgets.IntSlider(min=0, max= Nt -1, step=1, value=0)
widgets.jslink((play, 'value'), (slider, 'value'))
interact(FVM_plot,
         x=fixed(x),
         u=fixed(p_all),
         step = play)
widgets.HBox([slider])

Current variables values:
  D  [m^2/s]: 1.00
  L   [ m ]: 300.0
  Nx  [---]: 10.0
  T   [ s ]: 40000.0
  Nt  [---]: 200.0
  dx  [ m ]: 3.00e+01
  dt  [ s ]: 2.00e+02
Courant, C_N, direction x: 2.22e-01


interactive(children=(Play(value=0, description='step', max=199), Output()), _dom_classes=('widget-interact',)…

## Task 2: 2D Implementation

The code below is set up to solve the 2D diffusion equation. It works more or less in a similar way to the Wednesday assignment, although one difference is the boundary conditions and initial conditions. The problem is set up such that the initial condition is a "pulse" of the quantity of interest over a square domain in $x$ and $y$. The boundary conditions are set on the edge of the square via the list `pBC` such that the elements in the list specify the left, right, bottom and top boundary. As with the 1D case above, the boundary conditions are implemented in the finite volumes prior to looping over all the rest that do not fall on the boundary.

<div style="background-color:#AABAB2; color: black; vertical-align: middle; width:95%; padding:15px; margin: 10px; border-radius: 10px">
<p>
<b>Task 2:</b> 
    
Read through the code to identify the boundary conditions and initial conditions. Then derive the solution for the quantity of interest $\phi_i^{n+1}$ and implement it in the code.

</p>
</div>


In [4]:
def diffusion_2D(p, dx, dy, Nx, Ny, D, dt, pBC):
    """FVM diffusion, 2D.
    
    F_? refers to phi on the face of a volume.
    
    Other variable names should be obvious, based on previous assignment.
    """ 
    
    F_e = np.zeros((Nx, Ny))  
    F_w = np.zeros((Nx, Ny))  
    F_n = np.zeros((Nx, Ny))  
    F_s = np.zeros((Nx, Ny))  
    
    # Face values for boundary volumes
    F_w[0, :] = (pBC[0]-p[0,0])/(0.5*dx)
    F_e[-1, :] = (pBC[1]-p[-1,0])/(0.5*dx)
    F_n[:, -1] = (pBC[2]-p[-1,0])/(0.5*dy)
    F_s[:, 0] = (pBC[2]-p[-1,-1])/(0.5*dy)
    
    # Face values for interior volumes
    for i in range(1, Nx-1):
        for j in range(1, Ny-1):
            F_e[i, j] = (p[i+1,j] - p[i,j]) / (dx)
            F_w[i, j] = (p[i,j] - p[i-1,j]) / (dx)

    for i in range(1, Nx-1):
        for j in range(1, Ny-1):
            F_n[i, j] = (p[i,j+1] - p[i,j]) / (dx)
            F_s[i, j] = (p[i,j] - p[i,j-1]) / (dx)

    # compute phi for all volumes
    p_new = p + (D * dt / dx) * (F_e - F_w) + (D * dt / dx) * (F_n - F_s) 

    return p_new

def plot_2D(X, Y, u, step):
    fig = plt.figure()
    ax = fig.add_subplot(111,projection='3d')
    ax.plot_surface(X, Y, np.transpose(u[step]), cmap="Spectral")
    ax.axes.set_zlim3d(bottom=0, top=100) 
    ax.set_aspect('equalxy')
    ax.set_xlabel('Location x')
    ax.set_ylabel('Location y')
    ax.set_zlabel('$\phi$')
    plt.tight_layout()
    plt.show()
    
def check_variables_2D():
    print('Current variables values:')
    print(f'  D  [m^2/s]: {D:0.2f}')
    print(f'  Lx  [ m ]: {Lx:0.1f}')
    print(f'  Nx  [---]: {Nx:0.1f}')
    print(f'  Ly  [ m ]: {Ly:0.1f}')
    print(f'  Ny  [---]: {Ny:0.1f}')
    print(f'  T   [ s ]: {T:0.1f}')
    print(f'  Nt  [---]: {Nt:0.1f}')
    print(f'  dx  [ m ]: {dx:0.2e}')
    print(f'  dy  [ m ]: {dy:0.2e}')
    print(f'  dt  [ s ]: {dt:0.2e}')
    # print(f'Using central difference?: {central}')
    print(f'Solution shape p_all[t_i]: ({Nx}, {Ny})')
    print(f'Total time steps in p_all: {Nt+1}')
    print(f'Courant, C_N, direction x: {D*dt/dx**2:.2e}')
    print(f'Courant, C_N, direction y: {D*dt/dy**2:.2e}')

In [5]:
Lx = 500
Nx = 10
Ly = 500
Ny = 10

pBC = [0, 0, 0, 0]

D = 1

T = 5000
Nt = 500

dx = Lx/Nx
dy = Ly/Ny  
dt = T/Nt

x = np.linspace(dx/2, Lx-dx/2, Nx)
y = np.linspace(dy/2, Ly-dy/2, Ny) 

p_all = np.zeros((Nt+1, Nx, Ny))

# You can play with the boundary conditions if you wish!
p_all[0, 0, :] = pBC[0]                  
p_all[0, -1, :] = pBC[1] 
p_all[0, :, 0] = pBC[2]
p_all[0, :, -1] = pBC[3]

# You can also play with the initial conditions!
p_all[0, int(235/dx):int(265/dx + 1), int(235/dy):int(265/dy + 1)] = 100

check_variables_2D()

for t in range(Nt):
    p = diffusion_2D(p_all[t], dx, dy, Nx, Ny, D, dt, pBC)
    p_all[t+1] = p

X, Y = np.meshgrid(x, y)  

play = widgets.Play(min=0, max=Nt - 1, step=1, value=0, interval=100, disabled=False)
slider = widgets.IntSlider(min=0, max=Nt - 1, step=1, value=0)
widgets.jslink((play, 'value'), (slider, 'value'))

interact(plot_2D, X=fixed(X), Y=fixed(Y), u=fixed(p_all), step=play)
widgets.HBox([slider])

Current variables values:
  D  [m^2/s]: 1.00
  Lx  [ m ]: 500.0
  Nx  [---]: 10.0
  Ly  [ m ]: 500.0
  Ny  [---]: 10.0
  T   [ s ]: 5000.0
  Nt  [---]: 500.0
  dx  [ m ]: 5.00e+01
  dy  [ m ]: 5.00e+01
  dt  [ s ]: 1.00e+01
Solution shape p_all[t_i]: (10, 10)
Total time steps in p_all: 501
Courant, C_N, direction x: 4.00e-03
Courant, C_N, direction y: 4.00e-03


interactive(children=(Play(value=0, description='step', max=499), Output()), _dom_classes=('widget-interact',)…

**End of notebook.**
<h2 style="height: 60px">
</h2>
<h3 style="position: absolute; display: flex; flex-grow: 0; flex-shrink: 0; flex-direction: row-reverse; bottom: 60px; right: 50px; margin: 0; border: 0">
    <style>
        .markdown {width:100%; position: relative}
        article { position: relative }
    </style>
    <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">
      <img alt="Creative Commons License" style="border-width:; width:88px; height:auto; padding-top:10px" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" />
    </a>
    <a rel="TU Delft" href="https://www.tudelft.nl/en/ceg">
      <img alt="TU Delft" style="border-width:0; width:100px; height:auto; padding-bottom:0px" src="https://gitlab.tudelft.nl/mude/public/-/raw/main/tu-logo/TU_P1_full-color.png"/>
    </a>
    <a rel="MUDE" href="http://mude.citg.tudelft.nl/">
      <img alt="MUDE" style="border-width:0; width:100px; height:auto; padding-bottom:0px" src="https://gitlab.tudelft.nl/mude/public/-/raw/main/mude-logo/MUDE_Logo-small.png"/>
    </a>
    
</h3>
<span style="font-size: 75%">
&copy; Copyright 2023 <a rel="MUDE Team" href="https://studiegids.tudelft.nl/a101_displayCourse.do?course_id=65595">MUDE Teaching Team</a> TU Delft. This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.